In [2]:
# rotate and cut the pictures
from PIL import Image
import os

path = "/home/mw/"
for i in range(1,4):
    for rawImage in os.listdir(path + str(i) + "/"):
        img = Image.open(path + str(i) + "/" + rawImage) # read image from file system
        img = img.rotate(180) # rotate it 180 degrees
        box = (400, 360, 760, 720) # the region that the signal sleeves will be seen
        img = img.crop(box) # cut the region down, into a 360 by 360 image
        img.show()
        img.save(path + str(i) + "/" + rawImage) # replace the original picture
        print(i)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


In [4]:
''' strain a tensorflow model that sorts the images
(generated from gpt 3.5)
write me a pthon script that trains a tflite image-sorting model using tflite maker library, it sorts images with size 300 times 300 pixels into 3 groups, reads the samples from"/homw/mw/1" "/home/mw/2" and "/home/mw/3". Store the trained model into new.tflite
'''

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

TRAIN_DIR = '/home/mw'

# Define the model architecture
model = Sequential([
  Conv2D(32, (3, 3), activation='relu', input_shape=(300, 300, 3)),
  MaxPooling2D((2, 2)),
  Conv2D(64, (3, 3), activation='relu'),
  MaxPooling2D((2, 2)),
  Conv2D(128, (3, 3), activation='relu'),
  MaxPooling2D((2, 2)),
  Flatten(),
  Dense(128, activation='relu'),
  Dense(3, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Create the data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=30, zoom_range=0.2, shear_range=0.2)
train_generator = train_datagen.flow_from_directory(
  TRAIN_DIR,
  target_size=(300, 300),
  batch_size=32,
  class_mode='categorical')

# Train the model
model.fit(train_generator, epochs=10)

# Save the model to a TFLite file
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open('sleeves.tflite', 'wb').write(tflite_model)

Found 54 images belonging to 11 classes.
Epoch 1/10


InvalidArgumentError:  logits and labels must be broadcastable: logits_size=[32,3] labels_size=[32,11]
	 [[node categorical_crossentropy/softmax_cross_entropy_with_logits (defined at tmp/ipykernel_116/3841252677.py:37) ]] [Op:__inference_train_function_1677]

Function call stack:
train_function


In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

TRAIN_DIR = '/home/mw'

# Define the model architecture
model = Sequential([
  Conv2D(32, (3, 3), activation='relu', input_shape=(300, 300, 3)),
  MaxPooling2D((2, 2)),
  Conv2D(64, (3, 3), activation='relu'),
  MaxPooling2D((2, 2)),
  Conv2D(128, (3, 3), activation='relu'),
  MaxPooling2D((2, 2)),
  Flatten(),
  Dense(128, activation='relu'),
  Dense(3, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Create the data generators for training and validation
train_datagen = ImageDataGenerator(
  rescale=1./255,
  rotation_range=30,
  zoom_range=0.2,
  shear_range=0.2,
  validation_split=0.2)  # Add the validation split here
train_generator = train_datagen.flow_from_directory(
  TRAIN_DIR,
  target_size=(300, 300),
  batch_size=32,
  class_mode='categorical',
  subset='training')  # Use subset='training' for the training data
validation_generator = train_datagen.flow_from_directory(
  TRAIN_DIR,
  target_size=(300, 300),
  batch_size=32,
  class_mode='categorical',
  subset='validation')  # Use subset='validation' for the validation data

# Train the model
model.fit(train_generator, epochs=10, validation_data=validation_generator)

# Save the model to a TFLite file
converter = tf.compat.v1.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open('sleeves.tflite', 'wb').write(tflite_model)

Found 45 images belonging to 11 classes.
Found 9 images belonging to 11 classes.
Epoch 1/10


InvalidArgumentError:  logits and labels must be broadcastable: logits_size=[32,3] labels_size=[32,11]
	 [[node categorical_crossentropy/softmax_cross_entropy_with_logits (defined at tmp/ipykernel_116/3322874766.py:45) ]] [Op:__inference_train_function_3353]

Function call stack:
train_function


In [7]:
import tensorflow as tf
import numpy as np
import tflite_model_maker as tfl

# Define the path to the image samples
sample_paths = ["/home/mw/1", "/home/mw/2", "/home/mw/3"]

# Define the input and output sizes of the model
input_size = (300, 300)
output_size = 3

# Load the image data from the sample paths
data = tfl.ImageClassifierDataLoader.from_folder(sample_paths)
train_data, test_data = data.split(0.9)

# Define the model architecture
model = tfl.image_classifier.create(train_data, model_spec=tfl.specs.MobilenetV2Spec(), validation_data=test_data)

# Evaluate the model accuracy
accuracy = model.evaluate(test_data)

# Convert the model to TFLite format
tflite_model = model.export(export_format=tfl.ExportFormat.TFLITE)

# Save the TFLite model to disk
with open('new.tflite', 'wb') as f:
    f.write(tflite_model)

ModuleNotFoundError: No module named 'tflite_model_maker'

In [ ]:
'''
write me a pythpn script that trains a tflite image-sorting model, the model sorts the images into 3 groups, the sample images of the three groups are stored in"./1" "./2" and "./3", randomly named and in the form of jpg with size 300 times 300. You should store the trained model into ./new.tflite
'''

In [ ]:
import tensorflow as tf
import numpy as np
import os

# Set up parameters for training
batch_size = 32
num_epochs = 10
learning_rate = 0.001

# Define input and output paths for training data
train_dir = "./"
group1_dir = os.path.join(train_dir, "1")
group2_dir = os.path.join(train_dir, "2")
group3_dir = os.path.join(train_dir, "3")

# Extract filenames
group1_files = os.listdir(group1_dir)
group2_files = os.listdir(group2_dir)
group3_files = os.listdir(group3_dir)

# Create lists of image paths and labels
image_paths = []
labels = []
for path, label in [(group1_dir, 0), (group2_dir, 1), (group3_dir, 2)]:
    for filename in os.listdir(path):
        if filename.endswith(".jpg"):
            image_paths.append(os.path.join(path, filename))
            labels.append(label)

# Shuffle the data
shuffled_indices = np.random.permutation(len(image_paths))
image_paths = np.array(image_paths)[shuffled_indices]
labels = np.array(labels)[shuffled_indices]

# Define image preprocessing function
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [300, 300])
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image

# Define data loading and preprocessing function
def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    return preprocess_image(image), label

# Create dataset with loaded and preprocessed images and labels
dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
dataset = dataset.map(load_and_preprocess_image)
dataset = dataset.batch(batch_size)

# Define tflite model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(300, 300, 3)),
    tf.keras.layers.Conv2D(32, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(3, activation="softmax")
])

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss_fn, metrics=["accuracy"])

# Train the model
model.fit(dataset, epochs=num_epochs)

# Save the tflite model to disk
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("new.tflite", "wb").write(tflite_model)